### this code calculates differences of vegetation indices between 2017 and 2018


In [ ]:
import pandas as pd
import copy
import warnings
warnings.filterwarnings('ignore') #don't display warnings

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import h5py
from skimage import exposure
import gdal
import os
import osr

def list_dataset(name,node):
    
    """list_dataset lists the name and location of each dataset stored in an hdf5 file.
    --------
    See Also:
    --------
    ls_dataset: 
        Lists name, shape, and type of each dataset stored in an hdf5 file.
    --------
    Usage:
    --------
    f = h5py.File('NEON_D02_SERC_DP3_368000_4306000_reflectance.h5','r') 
    f.visititems(list_dataset)"""
    
    if isinstance(node, h5py.Dataset):
        print(name)

def ls_dataset(name,node):
    
    """ls_dataset lists the name, shape, and datatype of each dataset stored in 
    an hdf5 file.
    --------
    See Also
    --------
    list_dataset: 
        Lists name and location of each dataset stored in an hdf5 file
    Example:
    --------
    f = h5py.File('NEON_D02_SERC_DP3_368000_4306000_reflectance.h5','r') 
    f.visititems(ls_dataset)"""
    
    if isinstance(node, h5py.Dataset):
        print(node)



In [ ]:
# Define a function that will allow us to output GeoTIFF files.
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array,epsg):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(epsg)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
    

In [ ]:
# Define a functoin that transform raster to array
def raster2array(geotif_file):
    metadata = {}
    dataset = gdal.Open(geotif_file)
    metadata['array_rows'] = dataset.RasterYSize
    metadata['array_cols'] = dataset.RasterXSize
    metadata['bands'] = dataset.RasterCount
    metadata['driver'] = dataset.GetDriver().LongName
    metadata['projection'] = dataset.GetProjection()
    metadata['geotransform'] = dataset.GetGeoTransform()

    mapinfo = dataset.GetGeoTransform()
    metadata['pixelWidth'] = mapinfo[1]
    metadata['pixelHeight'] = mapinfo[5]

    metadata['ext_dict'] = {}
    metadata['ext_dict']['xMin'] = mapinfo[0]
    metadata['ext_dict']['xMax'] = mapinfo[0] + dataset.RasterXSize/mapinfo[1]
    metadata['ext_dict']['yMin'] = mapinfo[3] + dataset.RasterYSize/mapinfo[5]
    metadata['ext_dict']['yMax'] = mapinfo[3]

    metadata['extent'] = (metadata['ext_dict']['xMin'],metadata['ext_dict']['xMax'],
                          metadata['ext_dict']['yMin'],metadata['ext_dict']['yMax'])

    if metadata['bands'] == 1:
        raster = dataset.GetRasterBand(1)
        metadata['noDataValue'] = raster.GetNoDataValue()
        metadata['scaleFactor'] = raster.GetScale()

        # band statistics
        metadata['bandstats'] = {} #make a nested dictionary to store band stats in same 
        stats = raster.GetStatistics(True,True)
        metadata['bandstats']['min'] = round(stats[0],2)
        metadata['bandstats']['max'] = round(stats[1],2)
        metadata['bandstats']['mean'] = round(stats[2],2)
        metadata['bandstats']['stdev'] = round(stats[3],2)

        array = dataset.GetRasterBand(1).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float)
        array[array==int(metadata['noDataValue'])]=np.nan
        array = array/metadata['scaleFactor']
        return array, metadata

    elif metadata['bands'] > 1:
        print('More than one band ... need to modify function for case of multiple bands')

In [ ]:
import os, fnmatch
listOfFiles_2017 = os.listdir('../../Data/capstone/2017/NEON_D17_SJER_DP3_258000_4108000_VegIndices/')  
listOfFiles_2018 = os.listdir('../../Data/capstone/2018/NEON_D17_SJER_DP3_258000_4108000_VegIndices/')  
pattern = "*.tif"  


for entry_2017 in listOfFiles_2017:
    if fnmatch.fnmatch(entry_2017, pattern):
        for entry_2018 in listOfFiles_2018:
            if fnmatch.fnmatch(entry_2017,entry_2018):
                print ('../../Data/capstone/2017/NEON_D17_SJER_DP3_258000_4108000_VegIndices/'+entry_2017)
                print ('../../Data/capstone/2018/NEON_D17_SJER_DP3_258000_4108000_VegIndices/'+entry_2018)
                # read in the corresponding 2017 and 2018 indices
                index_2017, index_2017_md = raster2array('../../Data/capstone/2017/NEON_D17_SJER_DP3_258000_4108000_VegIndices/'+entry_2017)
                index_2018, index_2018_md = raster2array('../../Data/capstone/2018/NEON_D17_SJER_DP3_258000_4108000_VegIndices/'+entry_2018)
                
                diff_output = index_2017-index_2018
                
                # save classification results
                array2raster('../output/fig/'+entry_2017+'_difference.tif',
                             (index_2017_md['extent'][0],index_2017_md['extent'][3]),
                             1,-1,
                             np.array(diff_output,dtype=float),
                             32611)
                
                #print (entry_2017.split("_"))
            
            
            